# IoT Project

##### To get started

We need the following Python libraries:  
- Scapy: For parsing network packet traces
- IPy: For each IP addressing types (what is a local network address vs what is public network address, etcc)
- manuf: Look up a device manufacturer based on MAC address data

In [51]:
!pip install scapy IPy manuf pycurl StringIO

  Could not find a version that satisfies the requirement StringIO (from versions: )
No matching distribution found for StringIO


In [3]:
from scapy.all import *
from manuf import manuf

In [11]:
#help(scapy.all)

In [39]:
packets =rdpcap("./packet_capture.pcap")
p = manuf.MacParser(update=True)
# Example for testing Manufacturer look up based on MAC ID
p.get_all('84:38:35:5a:70:40')[0]

'Apple'

In [47]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scapy.all import *
from IPy import IP as IPy
import pprint

d= defaultdict(list)

columns = ['MAC ID', 'Manufacturer']
df = pd.DataFrame(columns=columns)

#for i in range(len(packets)):
for i in range(50):
    packet = packets[i]
    ## traffic patterns:
    # WAN to LAN: ignore for now
    # LAN to LAN/WAN: 
    if (packet.haslayer(IP) == 1):
        #print(packet.show())
        mac_id = packet["Ether.src"]
        IP_src = packet["IP.src"]
        IP_dst = packet["IP.dst"]
        if (IPy(IP_src).iptype() == "PRIVATE"):
            if (IPy(IP_dst).iptype() == "PUBLIC"):
                if mac_id not in d:
                    d[mac_id] = set([p.get_all(mac_id)[0],IP_dst])
                else:
                    d[mac_id].add(IP_dst)
        
    
    
    # if the device is in the dataframe
    #df.loc[i] = [mac_id, p.get_all(mac_id)[0]]
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(d)

defaultdict(<class 'list'>,
            {   '28:f0:76:31:d3:58': {   '104.154.127.3',
                                         '108.177.98.189',
                                         '172.217.164.106',
                                         'Apple'},
                '84:38:35:5a:70:40': {   '151.101.1.254',
                                         '162.125.7.3',
                                         '185.217.0.110',
                                         '216.58.195.243',
                                         'Apple'},
                '88:71:e5:d2:73:4b': {   '34.236.127.105',
                                         '52.46.132.50',
                                         'AmazonTe'}})


In [59]:
import pycurl
from io import StringIO

c = pycurl.Curl()
c.setopt(c.URL, 'https://ipinfo.io/216.58.195.243?token=4b22562280ac95')

e = io.BytesIO()
c.setopt(pycurl.WRITEFUNCTION, e.write)
c.perform()
c.close()
htmlString = e.getvalue().decode('UTF-8')
print(htmlString)

{
  "ip": "216.58.195.243",
  "hostname": "sfo03s06-in-f19.1e100.net",
  "city": "Mountain View",
  "region": "California",
  "country": "US",
  "loc": "37.4192,-122.0570",
  "postal": "94043",
  "org": "AS15169 Google LLC"
}


In [60]:
type(htmlString)

str